In [1]:
# Cargamos librerías
import pandas as pd 
import numpy as np
from pickle import TRUE

# para modelos
from sklearn.preprocessing import LabelEncoder,MinMaxScaler

# plots
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly.express as px

# manejo de datatime
import calendar
import datetime
from datetime import timedelta, datetime
from enum import IntEnum

# acceso a sistema
import sys
import os 

# GLOBAL_VARIABLES
from platform import python_version
CWD = os.getcwd()
DATA_PATH = os.path.join(CWD, "../data_dsmarket")

In [2]:
print("Working with these versions of libraries\n")
print(f"Numpy version {np.__version__}")
print(f"Pandas version {pd.__version__}")
print(f"Python version {python_version()}")

print("\nDirectorio actual\n",CWD)
print("\nDirectorio con conjunto de datos\n",DATA_PATH)

Working with these versions of libraries

Numpy version 1.24.1
Pandas version 1.5.3
Python version 3.8.16

Directorio actual
 /Users/nono/Downloads/nuclio/TFM/fase02_clusters

Directorio con conjunto de datos
 /Users/nono/Downloads/nuclio/TFM/fase02_clusters/../data_dsmarket


In [3]:
# Funciones

def datosGenerales(df):
    # Vemos un shape
    print("Su shape",df.shape);

    # vemos número de duplicados
    print("El número de duplicados es ",\
          df.duplicated().sum());
    
    # vemos recuento de nulos y si los hay, vemos su porcentaje
    return df.isnull().sum().apply(lambda row:
                                    f"{row} nulos, {round(row / df.shape[0]*100,2)} %"
                                       if row != 0 else f"{row} nulos ________")
    
def cambioFormato(dataset, atributos, tipo):
    dataset[atributos] = dataset[atributos].astype(tipo)

We upload the 3 datasets:

*   **df_sales** indicates the sales divided by day and location
*   **df_prices** indicates the products, its cathegory and the price that will depend on the week in which the product has been sold
*   **df_events** is a calendar that indicates the days, the week and the possible events

In [4]:
%%time
# Cargamos los ficheros csv

df_events = pd.read_csv(os.path.join(DATA_PATH, 'daily_calendar_with_events_new.csv'), sep=';')
df_prices = pd.read_csv(os.path.join(DATA_PATH,'item_prices.csv'), sep=',')
df_sales = pd.read_csv(os.path.join(DATA_PATH,'item_sales.csv'), sep=',')

CPU times: user 4.96 s, sys: 516 ms, total: 5.48 s
Wall time: 5.48 s


In [5]:
df_calendar = pd.read_csv(f'{DATA_PATH}/df_calendar_final_20230317.csv',sep=",")

## Trabajamos con el calendario limpio

In [6]:
df_calendar.drop('Unnamed: 0',axis=1,inplace=True)
df_calendar['date'] = pd.to_datetime(df_calendar['date'])

In [7]:
df_calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1913 entries, 0 to 1912
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           1913 non-null   datetime64[ns]
 1   mes            1913 non-null   int64         
 2   weekday        1913 non-null   object        
 3   weekday_int    1913 non-null   int64         
 4   d              1913 non-null   object        
 5   event          1913 non-null   object        
 6   festivos_eeuu  1913 non-null   object        
 7   year_false     1913 non-null   int64         
 8   week_false     1913 non-null   int64         
 9   yearweek       1913 non-null   int64         
dtypes: datetime64[ns](1), int64(5), object(4)
memory usage: 149.6+ KB


In [8]:
df_calendar.head(10)

,date,mes,weekday,weekday_int,d,event,festivos_eeuu,year_false,week_false,yearweek
0,2011-01-29,1,Saturday,1,d_1,No event,No event,2011,5,201105
1,2011-01-30,1,Sunday,2,d_2,No event,No event,2011,5,201105
2,2011-01-31,1,Monday,3,d_3,No event,No event,2011,5,201105
3,2011-02-01,2,Tuesday,4,d_4,No event,No event,2011,5,201105
4,2011-02-02,2,Wednesday,5,d_5,No event,No event,2011,5,201105
5,2011-02-03,2,Thursday,6,d_6,No event,No event,2011,5,201105
6,2011-02-04,2,Friday,7,d_7,No event,No event,2011,5,201105
7,2011-02-05,2,Saturday,1,d_8,No event,No event,2011,6,201106
8,2011-02-06,2,Sunday,2,d_9,SuperBowl,SuperBowl,2011,6,201106
9,2011-02-07,2,Monday,3,d_10,No event,No event,2011,6,201106


In [9]:
df_calendar['yearweek']=df_calendar['yearweek'].astype('object')

In [10]:
# Obtenemos las semanas y lo pasamos a una lista
semanas=df_calendar[['yearweek']].value_counts().to_frame().sort_values(by='yearweek').reset_index()
lista_semanas=semanas['yearweek'].to_list()

In [11]:
# comprobamos que tenemos la última semana incompleta
# Procederemos a borrarla
print(lista_semanas[-1]);
lista_semanas.pop()
print(lista_semanas[-1]);

201617
201616


In [12]:
# Agrupacion de ventas por semana. VOY DIRECTO ABAJO DE PRICES
# TENGO Q HACER EL PRODUCTO CARTESIANO ENTRE PRODUCTOS Y FECHAS
unique_id=df_sales['id'].unique()
len(unique_id)

30490

In [13]:
# Creamos el producto cartesiano
cartesian_product = pd.MultiIndex.from_product([lista_semanas, unique_id], names = ["week", "id"])
len(cartesian_product)

8323770

In [14]:
full_df = pd.DataFrame(index = cartesian_product).reset_index()

In [15]:
full_df.tail(3)

,week,id
8323767,201616,SUPERMARKET_3_825_PHI_3
8323768,201616,SUPERMARKET_3_826_PHI_3
8323769,201616,SUPERMARKET_3_827_PHI_3


## Transposición de las columnas
Paso siguiente: a las ventas ponerle fechas --> cruzarlo con calendar, y pasarlo a weeks (**resample**)
y despues lo cruzamos con este full_df(producto cartesiano) 


In [16]:
columns_cat = ['item',
               'category',
               'department',
               'store',
               'store_code',
               'region']

In [17]:
cambioFormato(df_sales, columns_cat, 'category')

In [18]:
%%time
# melt >> es útil para enviar mensajes a un DataFrame en un formato en el que una o más columnas son
# variables de identificación, mientras que todas las demás columnas, consideradas variables medidas,
# no están vinculadas al eje de la fila, dejando solo dos columnas sin identificador, variable y valor. 
### Resumiendo, deshacemos el pivot aplicando los identificadores y los pivotados. ###

# id_vars[tupla, lista o ndarray, opcional] : Columna(s) para usar como variables de identificación. 
# value_vars[tupla, lista o ndarray, opcional]: Columna(s) para anular el pivote. Si no se especifica,
# usa todas las columnas que no están configuradas como id_vars. 
# var_name[scalar]: Nombre a usar para la columna ‘variable’. Si es Ninguno, usa marco.columnas.nombre
# o ‘variable’. 
# value_name[scalar, default ‘value’]: Nombre a usar para la columna ‘value’. 
# col_level[int o string, opcional]: si las columnas son un índice múltiple, use este nivel para derretir. (#nono)

df_sales_melted = df_sales.melt(id_vars=['id',
                                         'item',
                                         'category',
                                         'department',
                                         'store',
                                         'store_code',
                                         'region'],
                                value_vars=df_sales.columns[7:],
                                var_name='d',
                                value_name='sales')

CPU times: user 7.82 s, sys: 1.4 s, total: 9.22 s
Wall time: 9.2 s


In [19]:
df_sales_melted.head(2)

,id,item,category,department,store,store_code,region,d,sales
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0


In [20]:
df_sales_melted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58327370 entries, 0 to 58327369
Data columns (total 9 columns):
 #   Column      Dtype   
---  ------      -----   
 0   id          object  
 1   item        category
 2   category    category
 3   department  category
 4   store       category
 5   store_code  category
 6   region      category
 7   d           object  
 8   sales       int64   
dtypes: category(6), int64(1), object(2)
memory usage: 1.7+ GB


In [21]:
df_sales_melted.head(4).T

,0,1,2,3
id,ACCESORIES_1_001_NYC_1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_003_NYC_1,ACCESORIES_1_004_NYC_1
item,ACCESORIES_1_001,ACCESORIES_1_002,ACCESORIES_1_003,ACCESORIES_1_004
category,ACCESORIES,ACCESORIES,ACCESORIES,ACCESORIES
department,ACCESORIES_1,ACCESORIES_1,ACCESORIES_1,ACCESORIES_1
store,Greenwich_Village,Greenwich_Village,Greenwich_Village,Greenwich_Village
store_code,NYC_1,NYC_1,NYC_1,NYC_1
region,New York,New York,New York,New York
d,d_1,d_1,d_1,d_1
sales,0,0,0,0


In [22]:
# Sin anotación científica
df_sales_melted['sales'].describe().to_frame().apply(lambda x: '%.5f' % x, axis=1)

count    58327370.00000
mean            1.12632
std             3.87311
min             0.00000
25%             0.00000
50%             0.00000
75%             1.00000
max           763.00000
dtype: object

### Trabajamos en el inner join de Calendario y  Ventas

In [23]:
df_calendar.head(2)

,date,mes,weekday,weekday_int,d,event,festivos_eeuu,year_false,week_false,yearweek
0,2011-01-29,1,Saturday,1,d_1,No event,No event,2011,5,201105
1,2011-01-30,1,Sunday,2,d_2,No event,No event,2011,5,201105


In [24]:
df_calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1913 entries, 0 to 1912
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           1913 non-null   datetime64[ns]
 1   mes            1913 non-null   int64         
 2   weekday        1913 non-null   object        
 3   weekday_int    1913 non-null   int64         
 4   d              1913 non-null   object        
 5   event          1913 non-null   object        
 6   festivos_eeuu  1913 non-null   object        
 7   year_false     1913 non-null   int64         
 8   week_false     1913 non-null   int64         
 9   yearweek       1913 non-null   object        
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 149.6+ KB


In [25]:
cambioFormato(df_calendar, ['weekday','d','event'], 'category')

In [26]:
# Comprobamos que las dataframes que vamos a unir tengan la misma
# última semana
print("df_calendar", df_calendar['yearweek'].max());

df_calendar 201617


In [27]:
df_calendar = df_calendar[df_calendar['yearweek'] != 201617]

In [28]:
print("df_calendar", df_calendar['yearweek'].max());

df_calendar 201616


In [29]:
%%time
# Merge con sales con ventas > 0
df_merged_inner = pd.merge(left=df_sales_melted,right=df_calendar, left_on='d', right_on='d')

CPU times: user 12.9 s, sys: 4.26 s, total: 17.1 s
Wall time: 18.3 s


In [30]:
df_merged_inner.drop('d',axis=1,inplace=True)

In [31]:
df_merged_inner['yearweek'].max()

201616

In [32]:
df_merged_inner['total_transacciones']=df_merged_inner['sales']

In [33]:
df_merged_inner.sample(5)

,id,item,category,department,store,store_code,region,sales,date,mes,weekday,weekday_int,event,festivos_eeuu,year_false,week_false,yearweek,total_transacciones
31877069,SUPERMARKET_3_599_BOS_1,SUPERMARKET_3_599,SUPERMARKET,SUPERMARKET_3,South_End,BOS_1,Boston,0,2013-12-09,12,Monday,3,No event,No event,2013,49,201349,0
13182311,HOME_&_GARDEN_2_389_NYC_4,HOME_&_GARDEN_2_389,HOME_&_GARDEN,HOME_&_GARDEN_2,Brooklyn,NYC_4,New York,0,2012-04-05,4,Thursday,6,No event,No event,2012,14,201214,0
9512692,SUPERMARKET_3_637_PHI_3,SUPERMARKET_3_637,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,0,2011-12-06,12,Tuesday,4,No event,No event,2011,49,201149,0
34917692,ACCESORIES_2_129_NYC_3,ACCESORIES_2_129,ACCESORIES,ACCESORIES_2,Tribeca,NYC_3,New York,1,2014-03-19,3,Wednesday,5,No event,No event,2014,11,201411,1
51397992,HOME_&_GARDEN_1_444_PHI_1,HOME_&_GARDEN_1_444,HOME_&_GARDEN,HOME_&_GARDEN_1,Midtown_Village,PHI_1,Philadelphia,0,2015-09-10,9,Thursday,6,No event,No event,2015,36,201536,0


In [34]:
df_merged_inner['id'].nunique()

30490

In [35]:
df_merged_inner.groupby('yearweek')['id'].size() #213430 = 30490 id * 7 dias de cada semana

yearweek
201105    213430
201106    213430
201107    213430
201108    213430
201109    213430
           ...  
201612    213430
201613    213430
201614    213430
201615    213430
201616    213430
Name: id, Length: 273, dtype: int64

In [36]:
df_merged_inner.groupby('yearweek')['id'].size().tail(4)
# Se decide borrar la última semana ya que se puede comprobar
# que tenemos falta de información a la hora de hacer el resample()
# ya que no estarían todos los id de productos para completarlo. 
# 201617     60980

yearweek
201613    213430
201614    213430
201615    213430
201616    213430
Name: id, dtype: int64

In [37]:
print(df_merged_inner.shape)
df_merged_inner = df_merged_inner[df_merged_inner['yearweek'] != 201617]
print(df_merged_inner.shape)

(58266390, 18)
(58266390, 18)


In [38]:
df_merged_inner['festivos_eeuu'].value_counts()

No event                   55217390
Pascua                       365880
San Valentin                 182940
Presidents Day               182940
San Patricio                 182940
SuperBowl                    182940
Halloween                    152450
Martin Luther King Jr        152450
NewYear                      152450
Navidad                      152450
Black Friday                 152450
Thanksgiving                 152450
Ramadan starts               152450
Labor day                    152450
dia de la independencia      152450
dia del padre                152450
Memorial day                 152450
dia de la madre              152450
Easter                        91470
amazon prime day              30490
Name: festivos_eeuu, dtype: int64

**Vamos a crear una variable de estación del año para poder analizar la variable en el armado de clusters.**

In [39]:
df_merged_inner['mes']=df_merged_inner['date'].dt.month

In [40]:
estaciones_mapping={
    1: 'invierno', 
    2: 'invierno', 
    3: 'invierno', 
    4: 'primavera', 
    5: 'primavera',
    6: 'primavera', 
    7: 'verano',
    8: 'verano', 
    9: 'verano', 
    10: 'otoño',
    11: 'otoño', 
    12: 'otoño'
} 
    

In [41]:
df_merged_inner['estacion']=df_merged_inner['mes'].map(estaciones_mapping)

In [58]:
from collections import Counter

def get_most_common(srs):
    x = list(srs)
    my_counter = Counter(x)
    return my_counter.most_common(1)[0][0]

In [60]:
%%time
## Resample
df_weekly_agg = df_merged_inner.groupby(['yearweek','id']).agg(
    {
        "sales":np.sum,
        "estacion":get_most_common,
        "festivos_eeuu": lambda x: x.value_counts().index[-1]
    }
).rename(
    columns = {
        "sales":"weekly_sales",
        "estacion":"estacion", 
        "festivos_eeuu":"evento"
    }
).reset_index()

CPU times: user 23min 40s, sys: 45.2 s, total: 24min 25s
Wall time: 23min 47s


In [61]:
df_weekly_agg.sample(25)

,yearweek,id,weekly_sales,estacion,evento
3016122,201251,SUPERMARKET_3_586_BOS_3,578,otoño,No event
5235432,201419,SUPERMARKET_2_338_BOS_3,0,primavera,dia de la madre
5136524,201416,HOME_&_GARDEN_2_325_NYC_2,1,primavera,Easter
300126,201114,SUPERMARKET_3_347_NYC_4,4,primavera,No event
5175628,201417,SUPERMARKET_3_057_PHI_2,0,primavera,No event
8121941,201610,HOME_&_GARDEN_2_064_BOS_2,0,invierno,No event
2401692,201231,SUPERMARKET_3_123_BOS_3,4,verano,No event
1544277,201203,SUPERMARKET_2_151_PHI_1,0,invierno,Martin Luther King Jr
3293696,201308,ACCESORIES_1_080_NYC_4,43,invierno,No event
1972844,201217,SUPERMARKET_2_322_NYC_2,0,primavera,No event


In [62]:
Q_datos_por_estacion=df_weekly_agg['estacion'].value_counts().to_frame()

In [63]:
Q_datos_por_estacion.head(4)

,estacion
invierno,2225770
primavera,2073320
verano,2012340
otoño,2012340


In [64]:
df_weekly_agg.tail(4)

,yearweek,id,weekly_sales,estacion,evento
8323766,201616,SUPERMARKET_3_827_NYC_4,10,primavera,No event
8323767,201616,SUPERMARKET_3_827_PHI_1,25,primavera,No event
8323768,201616,SUPERMARKET_3_827_PHI_2,13,primavera,No event
8323769,201616,SUPERMARKET_3_827_PHI_3,0,primavera,No event


In [65]:
## 273 semanas totales (recordar que borramos la última semana)
df_weekly_agg['yearweek'].nunique()

273

In [66]:
# Tomamos la información que necesitamos para mas tarde
add_info=df_merged_inner[['id',
                          'item',
                          'category',
                          'store',
                          'store_code',
                          'region']].drop_duplicates()

In [67]:
add_info.head(3)

,id,item,category,store,store_code,region
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,Greenwich_Village,NYC_1,New York
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,Greenwich_Village,NYC_1,New York
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,Greenwich_Village,NYC_1,New York


In [68]:
%%time
# Unimos el full_df que era el productor cartesiano de semanas y id

print(full_df.shape)

full_df = pd.merge(
    left = full_df,
    right = add_info,
    how = "left",
    on = "id"
)

print(full_df.shape)

(8323770, 2)
(8323770, 7)
CPU times: user 925 ms, sys: 127 ms, total: 1.05 s
Wall time: 1.05 s


In [69]:
full_df.rename(columns={'week':'yearweek'},inplace=True)
full_df.head(3)

,yearweek,id,item,category,store,store_code,region
0,201105,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,Greenwich_Village,NYC_1,New York
1,201105,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,Greenwich_Village,NYC_1,New York
2,201105,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,Greenwich_Village,NYC_1,New York


In [70]:
# vemos nulos
df_weekly_agg.isnull().sum()

yearweek        0
id              0
weekly_sales    0
estacion        0
evento          0
dtype: int64

In [71]:
full_df['yearweek'].max()

201616

In [72]:
df_weekly_agg['yearweek'].max()

201616

In [73]:
%%time
# Join por la izquierda.
print(full_df.shape)
full_df = pd.merge(full_df, df_weekly_agg, on = ['yearweek','id'], how = 'left')
print(full_df.shape)

(8323770, 7)
(8323770, 10)
CPU times: user 2.58 s, sys: 399 ms, total: 2.98 s
Wall time: 2.98 s


In [74]:
full_df.head(5)

,yearweek,id,item,category,store,store_code,region,weekly_sales,estacion,evento
0,201105,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,Greenwich_Village,NYC_1,New York,0,invierno,No event
1,201105,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,Greenwich_Village,NYC_1,New York,0,invierno,No event
2,201105,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,Greenwich_Village,NYC_1,New York,0,invierno,No event
3,201105,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,Greenwich_Village,NYC_1,New York,0,invierno,No event
4,201105,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,Greenwich_Village,NYC_1,New York,0,invierno,No event


In [75]:
full_df.isnull().sum()

yearweek        0
id              0
item            0
category        0
store           0
store_code      0
region          0
weekly_sales    0
estacion        0
evento          0
dtype: int64

In [76]:
# Ya con esta tabla, la tenemos que unir con la de precio asi sacamos el revenue

In [77]:
cambioFormato(df_prices,['item', 'category', 'store_code'],'category')

In [78]:
# vemos datos generales del dataset de precios
datosGenerales(df_prices)

Su shape (6965706, 5)
El número de duplicados es  212120


item             0 nulos ________
category         0 nulos ________
store_code       0 nulos ________
yearweek      243920 nulos, 3.5 %
sell_price       0 nulos ________
dtype: object

In [79]:
df_prices.dropna(inplace=True)

In [80]:
df_prices.sort_values(by='yearweek')

,item,category,store_code,yearweek,sell_price
5539160,SUPERMARKET_3_702,SUPERMARKET,PHI_1,201105.0,3.9360
6613947,HOME_&_GARDEN_2_445,HOME_&_GARDEN,PHI_3,201105.0,8.1000
261069,HOME_&_GARDEN_2_041,HOME_&_GARDEN,NYC_1,201105.0,6.5875
1516539,HOME_&_GARDEN_1_117,HOME_&_GARDEN,NYC_3,201105.0,11.2125
563249,SUPERMARKET_3_188,SUPERMARKET,NYC_1,201105.0,2.3760
...,...,...,...,...,...
2386874,HOME_&_GARDEN_2_338,HOME_&_GARDEN,NYC_4,201617.0,6.1500
2387152,HOME_&_GARDEN_2_339,HOME_&_GARDEN,NYC_4,201617.0,8.0750
6156810,SUPERMARKET_3_382,SUPERMARKET,PHI_2,201617.0,2.1360
2386079,HOME_&_GARDEN_2_334,HOME_&_GARDEN,NYC_4,201617.0,3.7250


In [81]:
# Eliminamos la semana 201617 por ser incompleta
df_prices = df_prices[df_prices['yearweek'] != 201617 ]

In [82]:
full_df['item'].nunique()

3049

## Final merge para tener 1 sola tabla

In [83]:
#%%time
print(full_df.shape)
df_final = pd.merge(full_df, df_prices, on = ['yearweek',
                                              'item',
                                              'category',
                                              'store_code'],
                    how = 'left')
print(df_final.shape)

(8323770, 10)
(8323770, 11)


In [84]:
df_final.isnull().sum()

yearweek              0
id                    0
item                  0
category              0
store                 0
store_code            0
region                0
weekly_sales          0
estacion              0
evento                0
sell_price      1757059
dtype: int64

In [85]:
%%time
# Debido a los nulos en precios, los completamos con los de la semana anterior
df_final["sell_price"] = df_final.groupby(["item",
                                           'category',
                                           'store_code'])["sell_price"].apply(lambda series:
                                                                              series.backfill().ffill())

<timed exec>:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)


CPU times: user 5.19 s, sys: 772 ms, total: 5.96 s
Wall time: 5.95 s


In [86]:
df_final['revenue']=df_final['weekly_sales']*df_final['sell_price']

In [87]:
df_final.sample(5)

,yearweek,id,item,category,store,store_code,region,weekly_sales,estacion,evento,sell_price,revenue
3138512,201302,HOME_&_GARDEN_1_536_PHI_3,HOME_&_GARDEN_1_536,HOME_&_GARDEN,Queen_Village,PHI_3,Philadelphia,0,invierno,Martin Luther King Jr,3.7250,0.0000
6199247,201451,HOME_&_GARDEN_1_068_NYC_4,HOME_&_GARDEN_1_068,HOME_&_GARDEN,Brooklyn,NYC_4,New York,10,otoño,Navidad,8.7125,87.1250
1237532,201145,SUPERMARKET_3_463_BOS_2,SUPERMARKET_3_463,SUPERMARKET,Roxbury,BOS_2,Boston,0,otoño,No event,7.1760,0.0000
2747483,201243,ACCESORIES_1_343_NYC_2,ACCESORIES_1_343,ACCESORIES,Harlem,NYC_2,New York,7,otoño,No event,7.0756,49.5292
1970968,201217,HOME_&_GARDEN_2_219_BOS_3,HOME_&_GARDEN_2_219,HOME_&_GARDEN,Back_Bay,BOS_3,Boston,11,primavera,No event,3.7125,40.8375


In [88]:
cambioFormato(df_final,['yearweek','weekly_sales'],np.int32)

In [89]:
datosGenerales(df_final)

Su shape (8323770, 12)
El número de duplicados es  0


yearweek        0 nulos ________
id              0 nulos ________
item            0 nulos ________
category        0 nulos ________
store           0 nulos ________
store_code      0 nulos ________
region          0 nulos ________
weekly_sales    0 nulos ________
estacion        0 nulos ________
evento          0 nulos ________
sell_price      0 nulos ________
revenue         0 nulos ________
dtype: object

In [90]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8323770 entries, 0 to 8323769
Data columns (total 12 columns):
 #   Column        Dtype   
---  ------        -----   
 0   yearweek      int32   
 1   id            object  
 2   item          category
 3   category      category
 4   store         category
 5   store_code    category
 6   region        category
 7   weekly_sales  int32   
 8   estacion      object  
 9   evento        object  
 10  sell_price    float64 
 11  revenue       float64 
dtypes: category(5), float64(2), int32(2), object(3)
memory usage: 492.3+ MB


In [91]:
df_final['sell_price'] = df_final['sell_price'].round(2)
df_final['revenue'] = df_final['revenue'].round(2)

In [92]:
df_final.to_csv(f'{DATA_PATH}/df_final_powerBI.csv')

In [ ]:
## df_lectura = pd.read_pickle(f'{DATA_PATH}/df_unido.pkl')